In [2]:
!pip install tensorflow[and-cuda]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 938.7 kB/s eta 0:00:0000:010:16m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.3/363.3 MB 1.1 MB/s eta 0:00:0000:0100:09m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 1.1 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.5/22.5 MB 1.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 964.3 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.7/895.7 kB 710.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.2/577.2 MB 1.1 MB/s eta 0:00:0000:0100:14m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 MB 1.2 MB/s eta 0:00:0000:0100:05
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 1.1 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 MB 1.1 MB/s eta 0:00:0000:0100:04m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.6/217.6 MB 1.1 MB/s eta 0:00:0000:0100:05m
   ━━━━━━━

In [20]:
!pip uninstall numpy

Found existing installation: numpy 2.1.3
Uninstalling numpy-2.1.3:
  Would remove:
    /home/group50/anaconda3/envs/condaenv/bin/f2py
    /home/group50/anaconda3/envs/condaenv/bin/numpy-config
    /home/group50/anaconda3/envs/condaenv/lib/python3.11/site-packages/numpy-2.1.3.dist-info/*
    /home/group50/anaconda3/envs/condaenv/lib/python3.11/site-packages/numpy.libs/libgfortran-040039e1-0352e75f.so.5.0.0
    /home/group50/anaconda3/envs/condaenv/lib/python3.11/site-packages/numpy.libs/libquadmath-96973f99-934c22de.so.0.0.0
    /home/group50/anaconda3/envs/condaenv/lib/python3.11/site-packages/numpy.libs/libscipy_openblas64_-ff651d7f.so
    /home/group50/anaconda3/envs/condaenv/lib/python3.11/site-packages/numpy/*
Proceed (Y/n)? ERROR: Operation cancelled by user
^C


In [22]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [23]:
DATASET_PATH = "VideoDataset2"
IMG_SIZE = (64, 64)
SEQUENCE_LENGTH = 20
NUM_CLASSES = 5
BATCH_SIZE = 8
EPOCHS = 30

In [24]:
class_labels = sorted(os.listdir(DATASET_PATH))
label_map = {label: idx for idx, label in enumerate(class_labels)}

FileNotFoundError: [Errno 2] No such file or directory: 'VideoDataset2'

In [ ]:
def video_to_frames(video_path, max_frames=SEQUENCE_LENGTH):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while len(frames) < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, IMG_SIZE)
        frame = frame.astype(np.float32) / 255.0
        frames.append(frame)
    cap.release()
    while len(frames) < max_frames:
        frames.append(np.zeros((IMG_SIZE[1], IMG_SIZE[0], 3)))
    return np.array(frames)

In [ ]:
X, y = [], []
print("\U0001F4E5 Loading video data...")
for label in class_labels:
    class_dir = os.path.join(DATASET_PATH, label)
    for file in tqdm(os.listdir(class_dir), desc=f"Processing {label}"):
        if file.endswith('.mp4'):
            video_path = os.path.join(class_dir, file)
            frames = video_to_frames(video_path)
            X.append(frames)
            y.append(label_map[label])

X = np.array(X)
y = to_categorical(np.array(y), num_classes=NUM_CLASSES)

📥 Loading video data...


Processing surprise: 100%|██████████| 81/81 [00:00<00:00, 152.36it/s]


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:

model = Sequential([
    TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(SEQUENCE_LENGTH, IMG_SIZE[0], IMG_SIZE[1], 3)),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Conv2D(64, (3, 3), activation='relu')),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Flatten()),
    LSTM(64),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDist  (None, 20, 62, 62, 32)    896       
 ributed)                                                        
                                                                 
 time_distributed_1 (TimeDi  (None, 20, 31, 31, 32)    0         
 stributed)                                                      
                                                                 
 time_distributed_2 (TimeDi  (None, 20, 29, 29, 64)    18496     
 stributed)                                                      
                                                                 
 time_distributed_3 (TimeDi  (None, 20, 14, 14, 64)    0         
 stributed)                                                      
                                                                 
 time_distributed_4 (TimeDi  (None, 20, 12544)         0

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stop]
)

Epoch 1/30
316/316 [==============================] - 81s 255ms/step - loss: 0.4621 - accuracy: 0.8340 - val_loss: 1.0079 - val_accuracy: 0.6630
Epoch 2/30
316/316 [==============================] - 81s 258ms/step - loss: 0.3454 - accuracy: 0.8831 - val_loss: 1.0105 - val_accuracy: 0.6899
Epoch 3/30
316/316 [==============================] - 83s 262ms/step - loss: 0.2928 - accuracy: 0.9021 - val_loss: 1.1387 - val_accuracy: 0.7104
Epoch 4/30
316/316 [==============================] - 82s 260ms/step - loss: 0.2055 - accuracy: 0.9283 - val_loss: 1.3346 - val_accuracy: 0.7184


In [ ]:
plot_model(model, to_file='model_layout.png', show_shapes=True, show_layer_names=True)

In [ ]:
model.save('emotion_cnn_lstm_model.h5')
print("✅ Model saved!")